# Laboratorio 7 - HMM y CRF con dataset real de NER en español
Francisco Castillo - 21562

## Ejercicio 1 - HMM:

### a) Instala e importa el dataset

In [182]:
from datasets import load_dataset
dataset = load_dataset("IEETA/SPACCC-Spanish-NER")

### b) Explora el dataset

In [183]:
train, test = dataset["train"], dataset["test"]
print(f"Train size: {len(train)}")
print(f"Test size: {len(test)}")

Train size: 33757
Test size: 11239


In [184]:
from pprint import pprint

pprint(train[:5])

{'ann_id': [0, 1, 2, 3, 4],
 'end_span': [73, 190, 278, 230, 305],
 'filename': ['S0004-06142005000500011-1',
              'S0004-06142005000500011-1',
              'S0004-06142005000500011-1',
              'S0004-06142005000500011-1',
              'S0004-06142005000500011-1'],
 'label': ['DISEASE', 'DISEASE', 'PROCEDURE', 'DISEASE', 'DISEASE'],
 'start_span': [50, 158, 192, 207, 280],
 'text': ['alergias medicamentosas',
          'fracturas vertebrales y costales',
          'intervenido de enfermedad de Dupuytren en mano derecha y by-pass '
          'iliofemoral izquierdo',
          'enfermedad de Dupuytren',
          'Diabetes Mellitus tipo II']}


El dataset no contiene etiquetas BIO predeterminadas, por lo que procedemos a crearlas manualmente. Es de notar que e `text` no se encuentra dentro de un corpus de texto, por lo que no hay palabras que no pertenezcan a una entidad nombrada; es decir, todas las palabras deben ser etiquetadas como parte de una entidad nombrada (B-XXX o I-XXX).

### c) Preprocesamiento

#### Creación de etiquetas BIO

In [185]:
def create_bio_labels(text, label):
    return [f"B-{label}"] + [f"I-{label}"] * (len(text.split()) - 1)

In [186]:
bio_tags = set()
for split in dataset.keys():
    for example in dataset[split]:
        bio_tags.update(create_bio_labels(example["text"], example["label"]))
bio_tags = sorted(bio_tags)  # sorted list of unique BIO tags
bio_tags = {tag: idx for idx, tag in enumerate(bio_tags)}  # mapping to integers

In [187]:
bio_tags

{'B-CHEMICAL': 0,
 'B-DISEASE': 1,
 'B-PROCEDURE': 2,
 'B-PROTEIN': 3,
 'B-SYMPTOM': 4,
 'I-CHEMICAL': 5,
 'I-DISEASE': 6,
 'I-PROCEDURE': 7,
 'I-PROTEIN': 8,
 'I-SYMPTOM': 9}

#### Mapear cada token y etiqueta a un número entero

In [188]:
all_words = list(train) + list(test)
all_word =  sorted(set(
    word
    for example in all_words
    for word in example["text"].split()
))
word2idx = {word: idx for idx, word in enumerate(sorted(all_word))}

In [189]:
def get_numbers(observation):
    text = observation["text"]
    label = observation["label"]
    return{
        "tokens": [word2idx[word] for word in text.split()],
        "bio_labels": [bio_tags[tag] for tag in create_bio_labels(text, label)]
    }

In [190]:
train = train.map(lambda x: get_numbers(x))
test = test.map(lambda x: get_numbers(x))

In [191]:
train[0]

{'filename': 'S0004-06142005000500011-1',
 'ann_id': 0,
 'label': 'DISEASE',
 'start_span': 50,
 'end_span': 73,
 'text': 'alergias medicamentosas',
 'tokens': [4195, 12111],
 'bio_labels': [1, 6]}

### d) Calculo de Marices

In [192]:
import numpy as np

n_tags = len(bio_tags)
n_words = len(word2idx)

# Initialize matrices
transition_counts = np.zeros((n_tags, n_tags), dtype=int)
emission_counts = np.zeros((n_tags, n_words), dtype=int)

# Count transitions and emissions
for example in train:
    labels = example["bio_labels"]
    tokens = example["tokens"]
    for i in range(len(labels)):
        emission_counts[labels[i], tokens[i]] += 1
        if i > 0:
            transition_counts[labels[i-1], labels[i]] += 1

# Normalize to get probabilities
transition_matrix = transition_counts / transition_counts.sum(axis=1, keepdims=True)
emission_matrix = emission_counts / emission_counts.sum(axis=1, keepdims=True)

In [193]:
# Handle NaN values (rows that sum to zero)
transition_matrix = np.nan_to_num(transition_matrix)
emission_matrix = np.nan_to_num(emission_matrix)

In [194]:
transition_matrix

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [195]:
emission_matrix.shape

(10, 17838)

### e) HMM Multinomial

In [196]:
import numpy as np
from hmmlearn.hmm import CategoricalHMM

In [197]:
idx2tag = {v: k for k, v in bio_tags.items()}
idx2word = {v: k for k, v in word2idx.items()}

In [198]:
# Helper: pack sequences into hmmlearn's expected shape
def pack_sequences(ds, key="tokens"):
    lengths = [len(ex[key]) for ex in ds]
    X = np.concatenate([np.asarray(ex[key], dtype=int).reshape(-1, 1) for ex in ds], axis=0)
    return X, lengths

In [199]:
# Compute start probabilities from first labels in each sequence
n_tags = len(bio_tags)
n_words = len(word2idx)
start_counts = np.zeros(n_tags, dtype=float)
for ex in train:
    if len(ex["bio_labels"]) > 0:
        start_counts[ex["bio_labels"][0]] += 1.0

In [200]:
# Laplace smoothing and normalization
eps = 1e-8
startprob = start_counts + eps
startprob = startprob / startprob.sum()

# (transition_counts: [n_tags, n_tags], emission_counts: [n_tags, n_words])
transmat = transition_counts.astype(float) + eps
transmat = transmat / np.clip(transmat.sum(axis=1, keepdims=True), 1.0, None)

emissionprob = emission_counts.astype(float) + eps
emissionprob = emissionprob / np.clip(emissionprob.sum(axis=1, keepdims=True), 1.0, None)

In [201]:
X_train, lengths_train = pack_sequences(train)

model = CategoricalHMM(
    n_components=n_tags,
    init_params="",  # do not overwrite our params
    params="",       # do not update during fit
    random_state=0
)
model.n_features = n_words
model.startprob_ = startprob
model.transmat_ = transmat
model.emissionprob_ = emissionprob

### f) Decodificación con Viterbi

In [202]:
all_correct = 0
all_total = 0
for ex in test:
    obs = np.asarray(ex["tokens"], dtype=int).reshape(-1, 1)
    y_pred = model.predict(obs)
    y_true = np.asarray(ex["bio_labels"], dtype=int)
    all_correct += (y_pred == y_true).sum()
    all_total += len(y_true)

print("Token accuracy (test set): {:.3f}".format(all_correct / max(all_total, 1)))

Token accuracy (test set): 0.811


### Reflexiona
#### ¿Cómo afecta la diversidad de datos a las matrices?

La diversidad de los datos influye directamente en las matrices de transición y emisión del HMM. Si algunas palabras o etiquetas aparecen muy pocas veces, las probabilidades estimadas para esas combinaciones serán muy bajas o incluso cero, lo que puede llevar al modelo a favorecer etiquetas frecuentes y a ignorar secuencias poco representadas. Además, la diversidad limitada puede causar que ciertas transiciones nunca se observen en el entrenamiento, generando huecos en la matriz de transición que afectan la capacidad del HMM para generalizar a nuevos ejemplos.

En este caso, hemos mencionado que no existen etiquetas `O` en el dataset, lo que significa que todas las palabras deben ser etiquetadas como parte de una entidad nombrada. Esto puede llevar a que el modelo tenga dificultades para manejar palabras que no pertenecen a ninguna entidad en un contexto real, ya que no ha aprendido a reconocerlas. La falta de diversidad en las etiquetas puede limitar la capacidad del modelo para adaptarse a situaciones del mundo real donde no todas las palabras son parte de entidades nombradas.

---

#### ¿Qué etiquetas o palabras presentan probabilidades bajas o cero y cómo podría resolverse?

Las palabras raras o únicas en el dataset, así como etiquetas poco frecuentes, tienden a tener emisiones o transiciones con probabilidades muy bajas o cero. Esto incluye nombres propios poco comunes, errores tipográficos o entidades que aparecen solo en unos pocos ejemplos. Además, palabras que aparecen en el test pero no en el entrenamiento (`OOV`) no tendrán representación en la matriz de emisión, lo que puede generar predicciones incorrectas o ceros. Esto, pues creamos un token para todas las palabras en el dataset (tanto train como test).

Para resolverlo, se puede aplicar Laplace Smoothing para evitar probabilidades nulas, introducir un token `<UNK>` para palabras desconocidas, agrupar etiquetas muy poco frecuentes en categorías generales y aumentar ejemplos de entidades raras. Además de, obviamente, incluir palabras con la etiqueta `O` en el dataset para que el modelo pueda aprender a manejarlas.

